In [1]:
import sys
import os
from pprint import pprint
sys.path.append(os.path.abspath("../../../"))
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display, HTML
from IPython.display import Markdown
import ipywidgets as iw
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
import usure.common.logging as usurelogging
from usure.config import config
from usure.classification.infrastructure import (
     BasicSentenceCleaner 
    ,CsvLabeledCommentsDao
    ,FileWordVectorsRep
    ,FileModelDao)
from usure.classification.core import SvmLab, ClassifierInput, LabReport, WordVectorsService, LabeledComments
import usure.classification.ui.utils as ui

Using TensorFlow backend.


In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = lines => false

<IPython.core.display.Javascript object>

In [4]:
pd.set_option('colheader_justify', 'left')
pd.set_option('display.max_colwidth', 200)

In [5]:
cleaner = BasicSentenceCleaner(config.assets)
comments_dao = CsvLabeledCommentsDao(config.sets, cleaner)
wv_rep = FileWordVectorsRep(config.embeddings)
model_dao = FileModelDao(config.models)
lab = SvmLab(model_dao)
comments = comments_dao.get("train.csv")
test_comments = comments_dao.get("test.csv")


In [6]:
def classification_lab_report(word_vector_no, input, lab):
    display(Markdown(f'## {word_vector_no}. Creating models with embeddings: {input.embeddings_name}'))
    lab_report = lab.train_by_stratifiedkfold(input, folds=10)
    return lab_report

def model_architecture_summary(lab_report):
    display(Markdown(f'### Mean Accuracies and losses'))
    display(lab_report.summary)
    display(Markdown(f'### Accuracies and losses by Stratified KFold (folds with balanced class distributions)'))
    model_reports_df = ui.model_reports_to_DataFrame(lab_report.model_reports)
    display(model_reports_df) 
    
def show_statistics_by_model(word_vector_no, word_vector_name, test_input, lab_report):
    """Shows statistics by models created by every folds"""
    display(Markdown(f'## {word_vector_no+1}. {word_vector_name}'))
    for fold_no, model_report in enumerate(lab_report.model_reports):
        display(Markdown(f'### Fold number: {fold_no+1}. Name: {model_report.name}'))
        ui.plot_learning_curves_if_epoches(model_report)
        display(Markdown(f'### Training'))
        print(model_report.training.classification_report)
        confu_mtx = model_report.training.confusion_matrix
        display(confu_mtx)
        display(Markdown(f'### Validation'))
        print(model_report.validation.classification_report)
        confu_mtx = model_report.validation.confusion_matrix
        display(confu_mtx)
        display(Markdown(f'### Test'))
        test_metrics, labeled_predictions = lab.test(model_report.name, test_input)
        print(f"Accuracy: {test_metrics.accuracy}")
        print(f"Loss: {test_metrics.loss}")
        print(test_metrics.classification_report)
        display(test_metrics.confusion_matrix)
        test_labeled_comments = test_input.labeled_comments
        labeled_comments = LabeledComments(f"{model_report.name}-{word_vector_name}.svm.csv",test_labeled_comments.comments ,labeled_predictions)
        comments_dao.save_from_origin(labeled_comments, "test.csv")

In [7]:
#%matplotlib notebook
display(Markdown(f'# Support vector machine'))
word_vectors = wv_rep.get_all()
word_vector_names = []
lab_reports = []

#word_vectors = [word_vectors.__next__()] #comment!!!!!
for i, word_vector in enumerate(word_vectors):
    #if "FBCR2013" in word_vector.name: #erase
    #    continue
    wv_service = WordVectorsService(word_vector)
    #test_inputs.append(ClassifierInput(test_comments, wv_service))
    input = ClassifierInput(comments, wv_service)
    lab_report = classification_lab_report(i+1, input, lab)
    model_architecture_summary(lab_report)
    #word_vector_names.append(word_vector.name)
    #lab_reports.append(lab_report)
    show_statistics_by_model(i, word_vector.name, ClassifierInput(test_comments, wv_service), lab_report)
'''   
for i, word_vector_name in enumerate(word_vector_names): 
    lab_report = lab_reports[i]
    show_statistics_by_model(i, word_vector_name, lab_report)
'''

# Support vector machine

## 1. Creating models with embeddings: CorpusFBCR2013.txt.usu.20_w.kvs

### Mean Accuracies and losses

,train_acc_mean,train_acc_stdev,val_acc_mean,val_acc_stdev
0,0.591959,0.004936,0.516337,0.02415


### Accuracies and losses by Stratified KFold (folds with balanced class distributions)

...

,N,NEU,NONE,P
True/Pred,,,,
N,45,14,22,33
NEU,0,0,0,0
NONE,0,0,0,0
P,0,0,0,0


### Test

Accuracy: 0.390745501285347
Loss: 1.4184624535147772
              precision    recall  f1-score   support

           N       0.39      1.00      0.56       456
         NEU       0.00      0.00      0.00       148
        NONE       0.00      0.00      0.00       224
           P       0.00      0.00      0.00       339

   micro avg       0.39      0.39      0.39      1167
   macro avg       0.10      0.25      0.14      1167
weighted avg       0.15      0.39      0.22      1167



,N,NEU,NONE,P
True/Pred,,,,
N,456,148,224,339
NEU,0,0,0,0
NONE,0,0,0,0
P,0,0,0,0


'   \nfor i, word_vector_name in enumerate(word_vector_names): \n    lab_report = lab_reports[i]\n    show_statistics_by_model(i, word_vector_name, lab_report)\n'

In [8]:
word_vector = wv_rep.get("SBW-vectors-300-min5-skipgram.bin")
wv_service = WordVectorsService(word_vector)
input = ClassifierInput(comments, wv_service)
lab_report = classification_lab_report(i+1, input, lab)
model_architecture_summary(lab_report)
show_statistics_by_model(i, word_vector.name, ClassifierInput(test_comments, wv_service), lab_report)

## 23. Creating models with embeddings: SBW-vectors-300-min5-skipgram.bin

### Mean Accuracies and losses

,train_acc_mean,train_acc_stdev,val_acc_mean,val_acc_stdev
0,0.391081,0.000263,0.391088,0.002379


### Accuracies and losses by Stratified KFold (folds with balanced class distributions)

,model name,train_acc,train_loss,val_acc,val_loss
0,77f0303b6d1d44ffbfb9c32dce1224cb,0.391221,1.238149,0.389831,1.323360
1,a93ac5c4884848b187ae98779e0c8c7b,0.391221,1.224852,0.389831,1.346273
2,2f995521f90544b49fe2ac4b6dfd3611,0.391221,1.226612,0.389831,1.335113
3,dd41802825bb41dda29710168af72fb0,0.390848,1.203139,0.393162,1.303305
4,240c270c971943489f50f147c9b9cb97,0.390848,1.269602,0.393162,1.395872
5,cf3cf8f2000a4ee0ab4a70e4f83e713d,0.390848,1.253225,0.393162,1.368907
6,e530252cd5cc4831aef6aaf78ab1e1ca,0.391429,1.252414,0.387931,1.417186
7,87474ba3320844eabaad9bcc5d6338f8,0.391429,1.222942,0.387931,1.311300
8,cdfb50ebdba34a1eb488bbf1758261bd,0.391056,1.350308,0.391304,1.447111
9,56365bd5bb3244c3b96349e8941bf34c,0.390684,1.287106,0.394737,1.396504


...

              precision    recall  f1-score   support

           N       0.39      1.00      0.57        45
         NEU       0.00      0.00      0.00        14
        NONE       0.00      0.00      0.00        22
           P       0.00      0.00      0.00        33

   micro avg       0.39      0.39      0.39       114
   macro avg       0.10      0.25      0.14       114
weighted avg       0.16      0.39      0.22       114



,N,NEU,NONE,P
True/Pred,,,,
N,45,14,22,33
NEU,0,0,0,0
NONE,0,0,0,0
P,0,0,0,0


### Test

Accuracy: 0.390745501285347
Loss: 1.3986251895414301
              precision    recall  f1-score   support

           N       0.39      1.00      0.56       456
         NEU       0.00      0.00      0.00       148
        NONE       0.00      0.00      0.00       224
           P       0.00      0.00      0.00       339

   micro avg       0.39      0.39      0.39      1167
   macro avg       0.10      0.25      0.14      1167
weighted avg       0.15      0.39      0.22      1167



,N,NEU,NONE,P
True/Pred,,,,
N,456,148,224,339
NEU,0,0,0,0
NONE,0,0,0,0
P,0,0,0,0
